In [12]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd

In [13]:
estados = 'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'

In [14]:
dadosuf = pd.read_excel('/content/drive/MyDrive/DataScience/estimativa_dou_2025.xls', header=1)
dadosuf = dadosuf.drop('Unnamed: 2', axis=1)
dadosuf.head()

,BRASIL E UNIDADES DA FEDERAÇÃO,POPULAÇÃO ESTIMADA
0,Brasil,213421037.0
1,Norte,18801282.0
2,Rondônia,1751950.0
3,Acre,884372.0
4,Amazonas,4321616.0


In [15]:
habuf = dadosuf.copy()
habuf.rename(columns={'BRASIL E UNIDADES DA FEDERAÇÃO':'UF', 'POPULAÇÃO ESTIMADA':'Populacao'}, inplace=True)
linhas_remover = [0, 1, 9, 19, 24, 28, 33, 34, 35, 36]
habuf = habuf.drop(linhas_remover)
habuf = habuf.sort_values(by='UF')
habuf = habuf.reset_index()
habuf = habuf.drop('index', axis=1)
habuf['Populacao'] = habuf['Populacao'].astype(int)
habuf.head()

,UF,Populacao
0,Acre,884372
1,Alagoas,3220848
2,Amapá,806517
3,Amazonas,4321616
4,Bahia,14870907


In [16]:
#Lendo múltiplos arquivos de um diretório.
diretorio = '/content/drive/MyDrive/DataScience/Covid-DadosBrasil'
covid_brasil = []
for nome_arquivo in os.listdir(diretorio):
  if nome_arquivo.endswith('.csv'):
    caminho_completo = os.path.join(diretorio, nome_arquivo)
    df = pd.read_csv(caminho_completo)
    covid_brasil.append(df)

In [17]:
#Concatenando Dataframes
covid_completo = pd.concat(covid_brasil, axis=1)
covid_completo.head()

,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos,...,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos,Ano,Casos Novos
0,2020,41620,2020,104818,2020,201013,2020,68201,2020,493400,...,2020,449674,2020,492583,2020,112505,2020,1462297,2020,90358
1,2021,46764,2021,137262,2021,232800,2021,58780,2021,777458,...,2021,1057443,2021,750952,2021,166025,2021,2993811,2021,145200
2,2022,70285,2022,93096,2022,191051,2022,55535,2022,498205,...,2022,1388454,2022,712618,2022,79751,2022,1859225,2022,124009
3,2023,7376,2023,8410,2023,13821,2023,4309,2023,54680,...,2023,194191,2023,100731,2023,7174,2023,447977,2023,13073
4,2024,3848,2024,5384,2024,5146,2024,5086,2024,23499,...,2024,67299,2024,39339,2024,2760,2024,153769,2024,9789


In [18]:
# Colocando a coluna 'Ano' como índice e colocando os nomes dos estados como cabeçalho.
ano_col = covid_completo.filter(like='Ano').iloc[:, 0]
casos_novos_col = covid_completo.filter(like='Casos Novos')
covid_final = pd.DataFrame(casos_novos_col.values, index=ano_col)
covid_final.columns = estados
covid_final.head()

,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
Ano,,,,,,,,,,,,,,,,,,,,,
2020,41620,104818,201013,68201,493400,335091,251701,248232,308868,200938,...,416566,434648,118083,95729,68689,449674,492583,112505,1462297,90358
2021,46764,137262,232800,58780,777458,621234,268110,381639,638836,169707,...,1181736,918176,269119,188897,60397,1057443,750952,166025,2993811,145200
2022,70285,93096,191051,55535,498205,480846,368252,681273,876276,117448,...,1262911,1348554,195416,187047,52159,1388454,712618,79751,1859225,124009
2023,7376,8410,13821,4309,54680,54483,39157,49728,160694,10744,...,118041,191241,13554,18617,5095,194191,100731,7174,447977,13073
2024,3848,5384,5146,5086,23499,26371,29287,28250,75844,2295,...,58301,87704,5801,14821,4064,67299,39339,2760,153769,9789


In [19]:
#Mudando formato e renomeando colunas do Dataframe para facilitar a junção com o Dataframe com dados da população total de cada estado.
covid_casos = covid_final.reset_index().melt(id_vars='Ano', var_name='UF', value_name='Casos')
covid_casos.head()

,Ano,UF,Casos
0,2020,AC,41620
1,2021,AC,46764
2,2022,AC,70285
3,2023,AC,7376
4,2024,AC,3848


In [20]:
#Criando dicionário com o nome completo dos estados para substituir pela abreviação nos Dataframes.
estados_completo = dict(zip(estados, habuf['UF']))
covid_casos['UF'] = covid_casos['UF'].map(estados_completo)
covid_casos.head()

,Ano,UF,Casos
0,2020,Acre,41620
1,2021,Acre,46764
2,2022,Acre,70285
3,2023,Acre,7376
4,2024,Acre,3848


In [21]:
#Unindo os dois Dataframes.
covid_merged = pd.merge(covid_casos, habuf, on='UF')
covid_merged.head()

,Ano,UF,Casos,Populacao
0,2020,Acre,41620,884372
1,2021,Acre,46764,884372
2,2022,Acre,70285,884372
3,2023,Acre,7376,884372
4,2024,Acre,3848,884372


In [23]:
#Calculando taxa de incidência da doença, mudando a métrica para cada 100k habitantes.
covid_100k = covid_merged.copy()
covid_100k['Incidencia(100k_Hab)'] = (covid_merged['Casos']/covid_merged['Populacao']*100000)
covid_100k['Incidencia(100k_Hab)'] = covid_100k['Incidencia(100k_Hab)'].astype(int)
covid_100k.head()

,Ano,UF,Casos,Populacao,Incidencia(100k_Hab)
0,2020,Acre,41620,884372,4706
1,2021,Acre,46764,884372,5287
2,2022,Acre,70285,884372,7947
3,2023,Acre,7376,884372,834
4,2024,Acre,3848,884372,435


In [24]:
estados_siglas = {
    'Acre':'AC',
    'Alagoas':'AL',
    'Amapa':'AP',
    'Amazonas':'AM',
    'Bahia':'BA',
    'Ceara':'CE',
    'Distrito Federal':'DF',
    'Espirito Santo':'ES',
    'Goias':'GO',
    'Maranhao':'MA',
    'Mato Grosso':'MT',
    'Mato Grosso do Sul':'MS',
    'Minas Gerais':'MG',
    'Parana':'PR',
    'Paraiba':'PB',
    'Para':'PA',
    'Pernambuco':'PE',
    'Piaui':'PI',
    'Rio Grande do Norte':'RN',
    'Rio Grande do Sul':'RS',
    'Rio de Janeiro':'RJ',
    'Rondonia':'RO',
    'Roraima':'RR',
    'Santa Catarina':'SC',
    'Sergipe':'SE',
    'Sao Paulo':'SP',
    'Tocantins':'TO'
}

In [25]:
covid_100k['UF'] = covid_100k['UF'].map(estados_siglas)
covid_100k

,Ano,UF,Casos,Populacao,Incidencia(100k_Hab)
0,2020,AC,41620,884372,4706
1,2021,AC,46764,884372,5287
2,2022,AC,70285,884372,7947
3,2023,AC,7376,884372,834
4,2024,AC,3848,884372,435
...,...,...,...,...,...
157,2021,TO,145200,1586859,9150
158,2022,TO,124009,1586859,7814
159,2023,TO,13073,1586859,823
160,2024,TO,9789,1586859,616


In [26]:
#Importando bibliotecas para gerar gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [27]:
#Gráfico do número de casos de Covid-19 por estado, no período de 2020 a 2025.
fig = px.line(covid_final, x=covid_final.index, y=covid_final.columns, markers=True, labels={'variable':'Estado'})
fig.update_layout(width=1300, height=600, font_family='DejaVu Sans', font_size=15, title_font_size=24,
                  title_text='Número de casos de COVID-19 por estado' + '<br><sup size=1 style="color:#555655">De 2020 a 2025</sup>',
                  xaxis_title='', yaxis_title='', plot_bgcolor='whitesmoke')
fig.show()

In [28]:
#Gráfico com a taxa de incidência de cada estado, a cada 100 mil habitantes.
fig = px.line(covid_100k, x='Ano', y='Incidencia(100k_Hab)', color='UF', markers=True)
fig.update_layout(width=1300, height=600, font_family='DejaVu Sans', font_size=15, title_font_size=24,
                  title='Evolução da Taxa de Incidência de COVID-19 por estado (por 100 mil habitantes)' +
                  '<br><sup size=1 style="color:#555655">De 2020 a 2025</sup>',
                  xaxis_title='',
                  yaxis_title='')
fig.show()